In [1]:
##################
#
#Test for comparison Analysis Software
#  Start date: 22 December 2011
#
##################

import wx
import wx.lib.agw.aui as aui
import wx.lib.scrolledpanel as scrolled
import wx.grid
import os
import sys, os.path
from textwrap import wrap
import copy
import numpy as np
import numpy.linalg as linalg
import xlrd
import xlwt
import urllib

#import matplotlib.pyplot

from matplotlib.backends.backend_wxagg import FigureCanvasWxAgg as FigureCanvas
from matplotlib.backends.backend_wxagg import NavigationToolbar2WxAgg
from matplotlib.figure import Figure
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
from matplotlib import rc
from matplotlib.mathtext import MathTextParser
mathtext_parser = MathTextParser("Bitmap")

In [3]:
# Set up some button numbers for the menu

ID_ABOUT=101
ID_OPEN=102
ID_SAVE=103
ID_BUTTON1=300
ID_EXIT=200

def gif(eq):
    page = "http://www.forkosh.com/mimetex.cgi?"+eq
    image = urllib.URLopener()
    image.retrieve(page,repr(os.path.dirname(os.path.realpath(sys.argv[0])))[1:-1]+'\\eqn.gif')


def mathtext_to_wxbitmap(eq):
    ftimage, depth = mathtext_parser.parse(eq, 150)
    return wx.BitmapFromBufferRGBA(
        ftimage.get_width(), ftimage.get_height(),
        ftimage.as_rgba_str())

In [4]:
class PageZero(scrolled.ScrolledPanel):
    # Set up front page, where user inputs the type of comparison and number of artefacts and participants
    # Allows user to change the number and to give the artefacts and partipants names
    
        def __init__(self, parent):
            scrolled.ScrolledPanel.__init__(self, parent)
            
            self.Data = ['Title','',1,2,['a_00'],['p_00','p_01'],[['1'],['1']]]

            # Request comparison information
            wx.StaticBox(self,-1,"",wx.Point(15,15),wx.Size(800,85))
            
            # Title
            self.lblTitl = wx.StaticText(self, -1, "Title of Comparison",wx.Point(20,30))
            self.editTitl = wx.TextCtrl(self, 20, self.Data[0], wx.Point(200, 30), wx.Size(270,-1))
            wx.EVT_TEXT(self,20,self.OnClick)
            
            # Type of comparison
            self.lblCompType = wx.StaticText(self, -1, "Type of Comparison",wx.Point(20,55))
            self.editCompType=wx.ComboBox(self, 23, "", wx.Point(200,55), wx.Size(100, -1),
                                          ('Primary','Linked'), wx.CB_DROPDOWN)
            #wx.EVT_COMBOBOX(self, 23, self.OnClick)
            wx.EVT_TEXT(self,23,self.OnClick)          

            # Input comparison structure 
            wx.StaticLine(self,-1,wx.Point(15,110),wx.Size(800,1)) 
            
            # Number of artefacts (accepts numbers between 1 and 50)
            self.lblNumArte = wx.StaticText(self, -1, "Number of Artefacts",wx.Point(30,133))
            self.editNumArte = self.sc = wx.SpinCtrl(self, 21, '',  wx.Point(180, 130), wx.Size(70,-1))
            self.editNumArte.SetRange(1, 50)
            self.editNumArte.SetValue(self.Data[2])
            wx.EVT_SPINCTRL(self,21,self.OnClick)
            #wx.TextCtrl(self, 21, str(self.Data[2]), wx.Point(210, 130), wx.Size(30,-1),wx.TE_PROCESS_ENTER )
            #wx.EVT_TEXT_ENTER(self,21,self.OnClick)

            # Number of participants (accepts numbers between 2 and 50)
            self.lblNumPart = wx.StaticText(self, -1, "Number of Participants",wx.Point(30,158))
            self.editNumPart = self.sc = wx.SpinCtrl(self, 22, '',  wx.Point(180, 155), wx.Size(70,-1))
            self.editNumPart.SetRange(2, 50)
            self.editNumPart.SetValue(self.Data[3])
            wx.EVT_TEXT(self,22,self.OnClick)

            self.NumParticipants = self.Data[3]
            self.NumArtefacts = self.Data[2]

            self.h = 205
            self.CreateInput()
            self.NAClicks = 0
            self.NPClicks = 0
            self.NCompTypeSelect = 0

            self.SetupScrolling()

        def CreateInput(self):
            self.lblArtID = wx.StaticText(self, -1, "Artefact identifiers", wx.Point(120,self.h-20))
            
            for i in range(0,self.NumArtefacts):
                # Provide correct number of input boxes for artefact values
                
                #vars(self)['Number'+str(i)] = wx.StaticText(self, -1, str(i+1),wx.Point(20+divmod(i,4)[1]*115,self.h+divmod(i,4)[0]*25))
                vars(self)['Number'+str(i)] = wx.StaticText(self, -1, str(i+1),wx.Point(105+i*115,self.h))   
                vars(self)['editArNa'+"%02d" % i] = wx.TextCtrl(self, 130+i, "a_"+"%02d" % i, wx.Point(120+i*115, self.h-3), wx.Size(80,-1))
                wx.EVT_TEXT(self, 130+i, self.OnClick)
                
            self.lblPartID = wx.StaticText(self, -1, "Participant",wx.Point(20,self.h+25))
            self.lblPartID2 = wx.StaticText(self, -1, "identifiers",wx.Point(20,self.h+40))
            self.lblPartM = wx.StaticText(self, -1, "Number of measurements",wx.Point(120,self.h+45))
            
            for i in range(0,self.NumParticipants):
                # Provide correct number of boxes for participant identifiers and measurement numbers 
                
                vars(self)['Number2'+str(i)] = wx.StaticText(self, -1, str(i+1),wx.Point(20,self.h+65+i*25))   
                vars(self)['editPaNa'+"%02d" % i] = wx.TextCtrl(self, 30+i, "p_"+"%02d" % i, wx.Point(35, self.h+65+i*25), wx.Size(80,-1))
                wx.EVT_TEXT(self, 30+i, self.OnClick)
                
                for j in range(0,self.NumArtefacts):
                    vars(self)['NumMeas'+"%02d" % i+"%02d" % j] = wx.TextCtrl(self, 230+10*i+j, self.Data[6][i][j], 
                                                                              wx.Point(120+j*115,self.h+65+i*25),wx.Size(80,-1))
                    wx.EVT_TEXT(self, 230+10*i+j, self.OnClick)


        def OnClick(self,event):
            # Allow artefact and participant identifiers to be changed, and the number of participants and artefacts to
            # be changed.
            
            for i in range(0,self.NumParticipants):
                
                if event.GetId() == 30+i:
                    # Participant names edited

                    #self.GetParent().EnableTab(3,False)
                    #self.GetParent().EnableTab(4,False)
                    self.Data[5][i]=vars(self)['editPaNa'+"%02d" % i].GetValue()
                    #self.GetParent().EnableTab(2,False)
                    
                for j in range(0,self.NumArtefacts):
                    if event.GetId() == 130+j:
                        
                        #self.GetParent().EnableTab(3,False)
                        #self.GetParent().EnableTab(4,False)
                        
                        # Artefact names edited
                        self.Data[4][j]=vars(self)['editArNa'+"%02d" % j].GetValue()
                        
                    if event.GetId() == 230+10*i+j:

                        self.GetParent().EnableTab(3,False)
                        self.GetParent().EnableTab(4,False)
                                   
                        # Measurement configuration edited  
                        self.Data[6][i][j]=vars(self)['NumMeas'+"%02d" % i+"%02d" % j].GetValue()

                if event.GetId() == 21:
                    # Event to change the number of artefacts
                    
                    self.DiffArte = self.editNumArte.GetValue() - self.NumArtefacts
                    #self.GetParent().EnableTab(3,False)
                    self.GetParent().EnableTab(4,False)                     

                    if self.DiffArte < 0 :
                        # If artefact number has decreased, remove previous input boxes for artefact identifiers 
                        for i in range(self.NumArtefacts+self.DiffArte,self.NumArtefacts):
                            vars(self)['Number'+str(i)].Destroy()
                            vars(self)['editArNa'+"%02d" % i].Destroy()
                            
                            for j in range(0,self.NumParticipants):
                                vars(self)['NumMeas'+"%02d" % j+"%02d" % i].Destroy()
                                
                        self.Data[6] = [self.Data[6][i][0:self.NumArtefacts+self.DiffArte] for i in range(0,self.NumParticipants)]
                        self.Data[4] = self.Data[4][0:self.NumArtefacts+self.DiffArte]
                        self.NumArtefacts = self.editNumArte.GetValue()
                        self.Data[2] = self.NumArtefacts


                    if self.DiffArte > 0 :
                        # If artefact number has increased, add input boxes for artefact identifiers 
                        for i in range(self.NumArtefacts,self.NumArtefacts+self.DiffArte):
                            vars(self)['Number'+str(i)] = wx.StaticText(self, -1, str(i+1),wx.Point(105+i*115,self.h))   
                            vars(self)['editArNa'+"%02d" % i] = wx.TextCtrl(self, 130+i, "a_"+"%02d" % i, 
                                                                            wx.Point(120+i*115, self.h-3), wx.Size(80,-1))
                            wx.EVT_TEXT(self, 130+i, self.OnClick)
                            
                            for j in range(0,self.NumParticipants):
                                vars(self)['NumMeas'+"%02d" % j+"%02d" % i] = wx.TextCtrl(self, 230+10*j+i, "0", 
                                                                    wx.Point(120+i*115,self.h+65+j*25),wx.Size(80,-1))
                                wx.EVT_TEXT(self, 230+10*j+i, self.OnClick)
                                    
                        [self.Data[6][i].extend(['0' for j in range(0,self.DiffArte)]) for i in range(0,self.NumParticipants)]
                        #[a[2][i].extend([0 for j in range(0,2)])for i in range(0,2)] 
                        self.Data[4].extend([vars(self)['editArNa'+"%02d" % i].GetValue() for i in range(self.NumArtefacts,self.NumArtefacts+self.DiffArte)])
                        self.NumArtefacts = self.editNumArte.GetValue()
                        self.Data[2] = self.NumArtefacts
                        self.GetParent().GetParent().GetParent().x = max(self.GetParent().GetParent().GetParent().x,240+self.NumArtefacts*115)
                        self.GetParent().SetClientSize(wx.Size(self.GetParent().GetParent().GetParent().x,self.GetParent().GetParent().GetParent().y))
                        self.SetupScrolling() 

                if event.GetId() == 22:
                    # Event to change number of participants

                    #self.GetParent().EnableTab(3,False)
                    self.GetParent().EnableTab(4,False)
                    self.DiffPart = self.editNumPart.GetValue() - self.NumParticipants
                    
                    if self.DiffPart != 0:
                        if self.NCompTypeSelect > 0:
                            if self.editCompType == 'Linked':
                                self.GetParent().EnableTab(2,False) 
                                
                    if self.DiffPart < 0 :
                        # If participant number has decreased, remove input boxes corresponding to fewer participants 
                        for j in range(self.NumParticipants+self.DiffPart,self.NumParticipants):
                            vars(self)['Number2'+str(j)].Destroy()
                            vars(self)['editPaNa'+"%02d" % j].Destroy()  
                            
                            for i in range(0,self.NumArtefacts):
                                vars(self)['NumMeas'+"%02d" % j+"%02d" % i].Destroy()
                                
                        self.Data[6] = self.Data[6][0:self.NumParticipants+self.DiffPart][:]
                        self.Data[5] = self.Data[5][0:self.NumParticipants+self.DiffPart]
                        
                    if self.DiffPart > 0 :
                        # If participant number increases, add new input boxes corresponding to extra number of participants 
                        for j in range(self.NumParticipants,self.NumParticipants+self.DiffPart):
                            vars(self)['Number2'+str(j)] = wx.StaticText(self, -1, str(j+1),wx.Point(20,self.h+65+j*25))   
                            vars(self)['editPaNa'+"%02d" % j] = wx.TextCtrl(self, 30+j, "p_"+"%02d" % j, 
                                                                            wx.Point(35, self.h+65+j*25), wx.Size(80,-1))

                            wx.EVT_TEXT(self, 30+j, self.OnClick)
                            
                            for i in range(0,self.NumArtefacts):
                                vars(self)['NumMeas'+"%02d" % j+"%02d" % i] = wx.TextCtrl(self, 230+10*j+i, "0", 
                                                                    wx.Point(120+i*115,self.h+65+j*25),wx.Size(80,-1))
                                wx.EVT_TEXT(self, 230+10*j+i, self.OnClick)
                                
                        self.Data[6].extend([['0' for j in range(0,self.NumArtefacts)] for i in range(0,self.DiffPart)])
                        self.Data[5].extend([vars(self)['editPaNa'+"%02d" % j].GetValue() for j in range(self.NumParticipants,self.NumParticipants+self.DiffPart)])

                        self.GetParent().GetParent().GetParent().y = max(self.GetParent().GetParent().GetParent().y,self.h+105+self.NumParticipants*25)
                        self.GetParent().GetParent().SetClientSize(wx.Size(self.GetParent().GetParent().GetParent().x,self.GetParent().GetParent().GetParent().y))
                        self.SetupScrolling() 

                    self.NumParticipants = self.editNumPart.GetValue()
                    self.Data[3] = self.NumParticipants

                if event.GetId() == 23:
                    # Event to trigger type of comparison decision

                    self.GetParent().EnableTab(3,False)
                    self.GetParent().EnableTab(4,False)
                    self.GetParent().EnableTab(1,True)
                    
                    if self.editCompType.GetValue() == 'Primary':
                        self.GetParent().EnableTab(2,True)
                        
                    if self.NCompTypeSelect > 0 and  self.editCompType.GetValue() != self.CurrentType:  
                        if self.editCompType.GetValue() == 'Linked':
                            self.GetParent().EnableTab(2,False)
                            
                    self.NCompTypeSelect = self.NCompTypeSelect + 1
                    self.CurrentType = self.editCompType.GetValue()
                    self.Data[1] = self.CurrentType

                if event.GetId() == 20:
                    self.Data[0] = self.editTitl.GetValue()  

In [5]:
class PageOne(scrolled.ScrolledPanel):
    # Set up first page where the user gives the weights of the participants in a primary comparison, and details of the 
    # linking for a linked comparison
    
    def __init__(self, parent):
        scrolled.ScrolledPanel.__init__(self, parent)
          
        Pg0 = self.GetParent().GetPage(0)            
        self.Page0PreviousData = copy.deepcopy(Pg0.Data)
        self.NSelections = 0
        self.Data = [[0.0 for m in range(0,Pg0.NumParticipants)],0,0]
        self.SetupScrolling()
             
    def PageChange(self):   
        Pg0 = self.GetParent().GetPage(0)
        self.PartList = Pg0.Data[5]

        if Pg0.Data[1] == 'Primary' and (self.Page0PreviousData[1] != Pg0.Data[1] or self.Page0PreviousData[3] != Pg0.Data[3] 
                                         or self.Page0PreviousData[5] != Pg0.Data[5] or self.NSelections == 0):
            for i in range(0,len(self.GetChildren())):
                self.GetChildren()[0].Destroy()

            self.Data = [[0 for m in range(0,Pg0.NumParticipants)],0,0]         
            wx.StaticBox(self,-1,"Input Participant Weights",wx.Point(15,15),wx.Size(800,Pg0.NumParticipants*25+45))
            self.ConstraintInstr = wx.StaticText(self, -1, "Please enter the weighting for each participant",wx.Point(30,30))
            
            for i in range(0,Pg0.NumParticipants):         
                vars(self)['Number3'+str(i)] = wx.StaticText(self, -1, self.PartList[i],wx.Point(45,55+i*25))  
                vars(self)['editWeights'+"%02d" % i] = wx.TextCtrl(self, 10+i, "", wx.Point(80, 52+i*25), wx.Size(80,-1))
                wx.EVT_TEXT(self, 10+i, self.On2Click)
                    
        if Pg0.Data[1] == 'Linked' and (self.Page0PreviousData[1] != Pg0.Data[1] or self.Page0PreviousData[3] != Pg0.Data[3] or
                                        self.Page0PreviousData[5] != Pg0.Data[5]  or self.NSelections ==0):
            self.PSelected = 0
            self.KSelected = 0
            
            for i in range(0,len(self.GetChildren())):
                self.GetChildren()[0].Destroy()
                
            self.Data = [1,[''],'']
            self.NLinks = self.Data[0]    
            wx.StaticBox(self,-1,"Input Linking Details",wx.Point(15,15),wx.Size(800,200))
            self.ConstraintInstr = wx.StaticText(self, -1, "Number of Linking Participants",wx.Point(30,30))
            self.NumList = [str(i) for i in range(1,Pg0.NumParticipants+1)]
            self.editNumLink = wx.ComboBox(self, 50, str(self.Data[0]), wx.Point(200,28), wx.Size(100, -1),
                                           self.NumList, wx.CB_DROPDOWN)
            wx.EVT_COMBOBOX(self, 50, self.On2Click)
            
            for i in range(1,2):  
                vars(self)['Link'+"%02d" % i] = wx.ComboBox(self,51+i,"",wx.Point(i*110,55), wx.Size(100,-1),self.PartList,
                                                            wx.CB_DROPDOWN)
                wx.EVT_COMBOBOX(self, 51+i, self.On2Click)

            self.lblLinkKCRV = wx.StaticText(self, -1, "KCRV from linking Comparison as Input parameter?",wx.Point(30,80))
            self.editLinkKCRV = wx.ComboBox(self, 90, 'No', wx.Point(400,78), wx.Size(100, -1),('No','Yes'), wx.CB_DROPDOWN)
            wx.EVT_COMBOBOX(self,90,self.On2Click)


        self.Page0PreviousData = copy.deepcopy(Pg0.Data)
        self.NSelections = self.NSelections +1
         
                       
    def On2Click(self,event):
        Pg0 = self.GetParent().GetPage(0)
        
        if Pg0.Data[1] == 'Linked':
            # Set up linking details
            
            if event.GetId() == 50:      
                if self.NLinks < int(self.editNumLink.GetValue()):
                       
                    self.GetParent().EnableTab(3,False)
                    self.GetParent().EnableTab(4,False)
                    
                    # Set up extra drop boxes to input linking labs    
                    for i in range(self.NLinks+1,int(self.editNumLink.GetValue())+1):  
                        vars(self)['Link'+"%02d" % i] = wx.ComboBox(self,51+i,"",wx.Point(i*110,55), wx.Size(100,-1),
                                                                    self.PartList,wx.CB_DROPDOWN)
                        wx.EVT_COMBOBOX(self, 51+i, self.On2Click)
                        
                    self.Data[1].extend(['' for i in range(0,int(self.editNumLink.GetValue())-self.NLinks)])  
                    
                if self.NLinks > int(self.editNumLink.GetValue()):
                    self.GetParent().EnableTab(3,False)
                    self.GetParent().EnableTab(4,False)
                    
                    for i in range(int(self.editNumLink.GetValue())+1,self.NLinks+1):  
                        vars(self)['Link'+"%02d" % i].Destroy()
                        
                    self.Data[1] = self.Data[1][0:self.NLinks]  
                    
                self.Data[0] = int(self.editNumLink.GetValue())
                self.NLinks = self.Data[0]
                    
            for i in range(1,self.NLinks+1):
                if event.GetId() == 51+i: 
                    self.GetParent().EnableTab(3,False)
                    self.GetParent().EnableTab(4,False)
                    
                    # Selection of linking participant has been made
                    self.Data[1][i-1] = vars(self)['Link'+"%02d" % i].GetValue()
                    self.PSelected = 1
                    
                    if self.KSelected == 1:
                        self.GetParent().EnableTab(2,True)                
                        self.GetParent().EnableTab(3,True)

                        
                if event.GetId() == 90:                     
                    self.GetParent().EnableTab(3,False)
                    self.GetParent().EnableTab(4,False)
                    
                    # Enable the measurement pane as KCRV chosen
                    self.Data[2] = self.editLinkKCRV.GetValue()
                    self.KSelected = 1
                    
                    if self.PSelected == 1:
                        self.GetParent().EnableTab(2,True)
                        self.GetParent().EnableTab(3,True)


                        
        if Pg0.Data[1] == 'Primary':
            # Allow weights of participants to be input for primary comparison
            
            for i in range(0,Pg0.NumParticipants):
                if event.GetId() == 10+i:
                    self.Data[0][i] = float(vars(self)['editWeights'+"%02d" % i].GetValue())
                    self.GetParent().EnableTab(4,False)

In [6]:
class PageTwo(scrolled.ScrolledPanel):
    # Set up second page where the user inputs the results and covariance matrix
    
    def __init__(self, parent):
        scrolled.ScrolledPanel.__init__(self, parent)

        Pg0 = self.GetParent().GetPage(0)            
        self.Page0PreviousData = copy.deepcopy(Pg0.Data)
        Pg1 = self.GetParent().GetPage(1)            
        self.Page1PreviousData = copy.deepcopy(Pg1.Data)
        self.NSelections = 0
        self.NCons = 0
        self.Data = [[],[],[]]
        self.SetupScrolling()
          
    def PageChange(self):
            
        Pg0 = self.GetParent().GetPage(0)
        Pg1 = self.GetParent().GetPage(1)
          

        if self.NSelections == 0 or Pg0.Data[1:7] != self.Page0PreviousData[1:7]: # Or the kcrv is now included from page 1
    
            self.GetParent().EnableTab(3,True)
        
            for i in range(0,len(self.GetChildren())):
                self.GetChildren()[0].Destroy()

                 
            self.NMeasurements = 0
            self.MeasurementTitles = []
            self.MeasurementValues = []
            self.CovarianceMatrix = []
            self.PartMeas = []
            n = 0
            
            # Set up matrix for user to input results
            for i in range(0,Pg0.NumParticipants):
                p = 0 
                for j in range(0,Pg0.NumArtefacts):
                    increment = int(vars(Pg0)['NumMeas'+"%02d" % i+"%02d" % j].GetValue())       
                               
                    self.MeasurementValues.extend(['0.0' for m in range(0,increment)])
                    self.MeasurementTitles.extend([[[''],[''],['']] for m in range(0,increment)])
                    [self.CovarianceMatrix[p].extend(['0.0' for m in range(0,increment)]) for p in range(0,self.NMeasurements)]
                    self.NMeasurements = self.NMeasurements + increment
                    self.CovarianceMatrix.extend([['0.0' for m in range(0,self.NMeasurements)] for p in range(0,increment)])
                    
                    for k in range(0,increment):                                      
                        self.MeasurementTitles[n][0] = vars(Pg0)['editPaNa'+"%02d" % i].GetValue() 
                        self.MeasurementTitles[n][1] = vars(Pg0)['editArNa'+"%02d" % j].GetValue()
                        self.MeasurementTitles[n][2] = str(k+1)
                        n += 1
                        p += 1
                        
                self.PartMeas.extend([p]) 
                
            self.CalculateCons()
            self.Data[0] = self.MeasurementTitles

            self.w = 110 + max([len(self.MeasurementTitles[n][0]+self.MeasurementTitles[n][1]+self.MeasurementTitles[n][2])
                                     for n in range(0,self.NMeasurements)])*9
            self.h = 85
                 
            #self.DrawTitles(0,self.NMeasurements)
            self.DrawMatrix(self.NMeasurements)
                 
              
        self.Page0PreviousData = copy.deepcopy(Pg0.Data)          
        self.Page1PreviousData = copy.deepcopy(Pg1.Data)         
        self.NSelections = self.NSelections + 1
        self.Data = [self.MeasurementTitles,self.MeasurementValues,self.CovarianceMatrix]

    def CalculateCons(self): 
        # Work out the number of constraints as inputs
          
        Pg0 = self.GetParent().GetPage(0)
        Pg1 = self.GetParent().GetPage(1)
          
        n = self.NMeasurements
        self.NCons = 0
        
        if Pg0.Data[1] == 'Linked':
            self.MeasurementValues.extend(['0.0' for m in range(0,int(Pg1.editNumLink.GetValue()))])
            self.MeasurementTitles.extend([[[''],[''],['']] for m in range(0,int(Pg1.editNumLink.GetValue()))])
            [self.CovarianceMatrix[i].extend(['0.0' for j in range(0,int(Pg1.editNumLink.GetValue()))]) for i in range(0,
                                                                                                        self.NMeasurements)]
            self.NMeasurements = self.NMeasurements + int(Pg1.editNumLink.GetValue())       
            self.CovarianceMatrix.extend([['0.0' for j in range(0,self.NMeasurements)] for i in range(0,
                                                                                            int(Pg1.editNumLink.GetValue()))])
            self.NCons = self.NCons + int(Pg1.editNumLink.GetValue())
            
            for k in range(1,int(Pg1.editNumLink.GetValue())+1):
                self.MeasurementTitles[n][0] = vars(Pg1)['Link'+"%02d" % k].GetValue()
                self.MeasurementTitles[n][1] = "Difference"
                self.MeasurementTitles[n][2] = ""
                n += 1
                    
                if Pg1.editLinkKCRV.GetValue() == 'Yes':
                    self.MeasurementValues.extend(['0.0'])    
                    self.MeasurementTitles.extend([[[''],['']]])    
                    [self.CovarianceMatrix[i].extend(['0.0']) for i in range(0,self.NMeasurements)]
                    self.NMeasurements = self.NMeasurements + 1
                    self.CovarianceMatrix.extend([['0.0' for j in range(0,self.NMeasurements)]])
                    self.NCons += 1
                    self.MeasurementTitles[n][0] = "Linking KCRV"
                    self.MeasurementTitles[n][1] = ""
                    self.MeasurementTitles[n][2] = ""         
            
    def DrawTitles(self,a,b):
        # Draw titles and measurement data input from a to b
        Pg0 = self.GetParent().GetPage(0)
        Pg1 = self.GetParent().GetPage(1)
        
        for i in range(a,b):                 
            vars(self)['MeasTitle1'+str(i)] = wx.StaticText(self, -1, self.MeasurementTitles[i][0],wx.Point(self.w+i*70+5,30))
            vars(self)['MeasTitle2'+str(i)] = wx.StaticText(self, -1, self.MeasurementTitles[i][1],wx.Point(self.w+i*70+5,45))
            vars(self)['MeasTitle3'+str(i)] = wx.StaticText(self, -1, self.MeasurementTitles[i][2],wx.Point(self.w+i*70+5,60))
            vars(self)['MeasTitle4'+str(i)] = wx.StaticText(self, -1, self.MeasurementTitles[i][0]+" "+
                                            self.MeasurementTitles[i][1]+" "+self.MeasurementTitles[i][2],
                                                            wx.Point(110,self.h+25*i))
            vars(self)['MeasValue'+"%02d" % i] = wx.TextCtrl(self, i, '', wx.Point(20,self.h+25*i-3), wx.Size(80,-1))
            #vars(self)['MeasValue'+"%02d" % i].SetValue('0.0')
            
            wx.EVT_TEXT(self,i,self.On3Click)
            
            if i == self.NMeasurements-1 and Pg0.editCompType.GetValue() == 'Linked':
                if Pg1.editLinkKCRV.GetValue() == 'Yes':
                    vars(self)['MeasValue'+"%02d" % i].Destroy()
                    self.ValueKCRV = wx.StaticText(self, -1, "0.0", wx.Point(45,self.h+25*i), wx.Size(60,-1))

    def DrawMatrix(self,a):
        # Draw titles measurement data input from a

        Pg0 = self.GetParent().GetPage(0)
        Pg1 = self.GetParent().GetPage(1)
        self.MTitle = wx.StaticText(self, -1, 'Measurement Values',wx.Point(10,10))
        self.CTitle = wx.StaticText(self, -1, 'Covariance Matrix',wx.Point(self.w,10))
        print(a)
        
        self.myGrid = wx.grid.Grid(self,200,wx.Point(self.w+70,self.h+25),wx.Size(500,500))
        self.myGrid.CreateGrid(a+1,a+1)
        #self.myGrid.SetDefaultRowSize(35,resizeExistingRows=False)
        #self.myGrid.SetDefaultColSize(80,resizeExistingCols=False)
        self.myGrid.SetRowLabelValue(0, "")
        self.myGrid.SetColLabelValue(0, "")
        self.myGrid.SetCellValue(0,0,"Measurement \r Values")
        self.myGrid.SetCellValue(0,1,"Covariance \r Matrix")
        self.myGrid.SetReadOnly(0,0,isReadOnly=True)
        self.Bind(wx.grid.EVT_GRID_CELL_CHANGED, self.OnChange)
        
        for i in range(a):
            self.myGrid.SetRowLabelValue(i+1,self.MeasurementTitles[i][0]+", "+self.MeasurementTitles[i][1]+" \r"+self.MeasurementTitles[i][2])
            self.myGrid.SetColLabelValue(i+1,self.MeasurementTitles[i][0]+", "+self.MeasurementTitles[i][1]+" \r"+self.MeasurementTitles[i][2])
            self.myGrid.SetReadOnly(0,i+1,isReadOnly=True)
            self.myGrid.SetCellValue(i+1,0,'0.0')
            
            for j in range(a):
                self.myGrid.SetCellEditor(i, j, wx.grid.GridCellFloatEditor()) 
                self.myGrid.SetCellValue(i+1,j+1,'0.0')   
                
            if i == self.NMeasurements-1 and Pg0.editCompType.GetValue() == 'Linked':
                if Pg1.editLinkKCRV.GetValue() == 'Yes':
                    self.myGrid.SetReadOnly(i+1,0,isReadOnly=True)

                      

                   
        self.GetParent().GetParent().GetParent().x = max(self.GetParent().GetParent().GetParent().x,1*100+a+100)
        self.GetParent().GetParent().GetParent().y = max(self.GetParent().GetParent().GetParent().y,self.h+25*a-3+100)
        self.GetParent().GetParent().SetClientSize(wx.Size(self.GetParent().GetParent().GetParent().x,
                                                           self.GetParent().GetParent().GetParent().y))
        self.SetupScrolling()              
            
    def OnChange(self,event):
        row = event.GetRow()
        col = event.GetCol()

        if col == 0 :
            self.myGrid.SetCellValue(row, col, self.myGrid.GetCellValue(row,col))
            self.MeasurementValues[row-1] = (self.myGrid.GetCellValue(row,col))
            self.GetParent().EnableTab(4,False)

        else:
            self.myGrid.SetCellValue(row, col, self.myGrid.GetCellValue(row,col))
            self.CovarianceMatrix[row-1][col-1] = self.myGrid.GetCellValue(row,col)
            self.GetParent().EnableTab(4,False)                      

        self.Data = [self.MeasurementTitles,self.MeasurementValues,self.CovarianceMatrix]

In [7]:
class PageThree(scrolled.ScrolledPanel):
    # Set up third page where the matrices are shown
    
    def __init__(self, parent):
        scrolled.ScrolledPanel.__init__(self, parent)
        #self.SetBackgroundColour('blue')

        Pg0 = self.GetParent().GetPage(0)
        Pg1 = self.GetParent().GetPage(1)
        Pg2 = self.GetParent().GetPage(2)
        self.box = wx.BoxSizer(wx.VERTICAL)
        self.SetSizer(self.box)
        
        # Set up error messages
        self.RealReq = wx.MessageDialog( self, " Please make sure that the values \n input on the 'Measurement' tab are real numbers.",
                                     "Real Numbers Required", wx.OK)
        self.RankDeficiency = wx.MessageDialog( self, " The design matrix relating to the comparision structure is rank deficient. \n The comparison structure may not be fully linked \n or a particular participant or artefact has no associated measurements.",
                                     "Rank Deficient Design Matrix", wx.OK)
        self.InternetReq = wx.MessageDialog( self, " It seems you do not have an internet connection. \n The matrices may not display correctly.  You can check them in the saved excel file if so. ",
                                     "Internet Connection Required for display", wx.OK)
        self.CovMatError = wx.MessageDialog( self, " The covariance matrix is not correctly constructed. \n It is not positive semi-definite. ",
                                     "Covariance Matrix Error", wx.OK)
        self.WeightsError = wx.MessageDialog( self, " The weights do not add up to 1. ",
                                     "Weights Error", wx.OK)
        self.Link = 0
        self.s = 0

    def PageChange(self):
        for i in range(0,len(self.GetChildren())):
            self.GetChildren()[0].Destroy()
        
        self.Link = 0

        Pg0 = self.GetParent().GetPage(0)
        Pg1 = self.GetParent().GetPage(1)
        Pg2 = self.GetParent().GetPage(2)
        
        if Pg0.editCompType.GetValue() == 'Linked':
            if Pg1.editLinkKCRV.GetValue() == 'Yes':
                self.Link = 1 
            
        self.L = Pg0.NumArtefacts + Pg0.NumParticipants + self.Link
        self.X = np.matrix(np.zeros([Pg2.NMeasurements,self.L],dtype = 'float'))

        # Iterate through the number of measurements, and add 1's in the right places in X
        for i in range(0,Pg2.NMeasurements):
            
            # Iterate through the number of artefacts
            for j in range(0,Pg0.NumArtefacts):
                if Pg2.MeasurementTitles[i][1] == Pg0.Data[4][j]:
                    self.X[i,j] = 1.0
                    
            # Iterate through the number of participants
            for k in range(0,Pg0.NumParticipants):
                if Pg2.MeasurementTitles[i][0] == Pg0.Data[5][k]:
                    self.X[i,Pg0.NumArtefacts+k] = 1.0
                    
            if Pg0.editCompType.GetValue() == 'Linked':
                if Pg1.editLinkKCRV.GetValue() == 'Yes' and (Pg2.MeasurementTitles[i][1] == 'Difference' or i == Pg2.NMeasurements -1):
                    self.X[i,Pg0.NumArtefacts+Pg0.NumParticipants] = -1.
                    
                    if i == Pg2.NMeasurements -1:
                        self.X[i,Pg0.NumArtefacts+Pg0.NumParticipants] = 1.
       
        try:
            # Put values into vector and covariances into a matrix
            self.y = np.matrix([[float(Pg2.Data[1][i])] for i in range(0,Pg2.NMeasurements)])
            self.CovMatrix = np.matrix([[float(Pg2.Data[2][i][j]) for i in range(0,Pg2.NMeasurements)] for j in range(0,Pg2.NMeasurements)])
            
        except ValueError:
            # Error if numbers are not real
            self.RealReq.ShowModal()
            
        else:
            self.str = r'$\left[\begin{array}{c}' 
            
            for i in range(0,len(self.y)):
                self.str = self.str+"%1.4g" % self.y[i]+ r'\\'
                
            self.str = self.str+ r'\end{array}\right]'
            self.str = self.str + r'=\left[\begin{array}{'
            
            for i in range(0,np.shape(self.X)[1]):
                self.str = self.str + 'r'
                
            self.str = self.str + '}'
            
            for i in range(0,np.shape(self.X)[0]):
                for j in range(0,np.shape(self.X)[1]):
                    self.str = self.str+str(int(self.X[i,j])) 
                        
                    if j < np.shape(self.X)[1]-1:
                        self.str = self.str+ '&'
                            
                self.str = self.str+ r'\\'
                
            self.str = self.str+ r'\end{array}\right]\left[\begin{array}{c}'
            
            for i in range(0,np.shape(self.X)[1]):
                if i < Pg0.NumArtefacts:
                    self.str = self.str+r'\theta_{'+str(Pg0.Data[4][i])+r'}\\'
                    
                elif i < Pg0.NumArtefacts + Pg0.NumParticipants:
                    self.str = self.str+r'\Delta_{'+str(Pg0.Data[5][i-Pg0.NumArtefacts])+r'}\\'
                    
                else:
                    self.str = self.str+r'KCRV_{link}\\'
                    
            self.str = self.str+r'\end{array}\right]+\mathrm{\varepsilon}$'

            try:
                gif(self.str)
                bmp1 = wx.Image(repr(os.path.dirname(os.path.realpath(sys.argv[0])))[1:-1]+"\\eqn.gif", wx.BITMAP_TYPE_ANY).ConvertToBitmap()
                T1 = wx.StaticText(self, -1, 'Please check that the matrices below reflect the structure of the comparison correctly. \n See the manual for more information',wx.Point(30,20))
                self.bitmap1 = wx.StaticBitmap(self, -1, bmp1, (30, 50))
                
            except:
                if self.s == 0:
                    self.InternetReq.ShowModal()
                    self.s = 1  
                    
                rc('text', usetex=False)
                self.figure = Figure(figsize = (8,5))
                self.canvas = FigureCanvas(self, -1, self.figure)
                self.axes = self.figure.add_axes([0.05,0.05,0.9,0.85])
                        
                self.axes.grid(False)
                self.axes.axis([0,1,-1,0])
                self.axes.xaxis.set_visible(False)
                self.axes.yaxis.set_visible(False)
                self.axes.set_title('Please check that the matrices below reflect the structure of the comparison correctly. \n See the manual for more information',fontsize = 12)
                
                for i in range(0,len(self.y)):
                    self.axes.text(0.1,-0.2+(i+1)*(-0.03),r'$'+"%1.4g" % self.y[i] +'$',fontsize = 12)
                                   
                self.axes.text(0.2,-0.3,'=',fontsize = 12) 
                
                for i in range(0,np.shape(self.X)[0]):
                    for j in range(0,np.shape(self.X)[1]):
                        self.axes.text(0.25+(j)*0.025,-0.2+(i+1)*(-0.03),r'$'+"%1d" % self.X[i,j] +'$',fontsize = 12)

                for i in range(0,np.shape(self.X)[1]):
                    if i < Pg0.NumArtefacts:
                        #text = '\theta_'+str(Pg1.Data[4][i]
                        self.axes.text(0.3+(np.shape(self.X)[1])*0.025,-0.2+(i+1)*(-0.03), r'$\theta_{'+str(Pg0.Data[4][i])+'}$',fontsize = 12)
                        
                    elif i < Pg0.NumArtefacts + Pg0.NumParticipants:
                        self.axes.text(0.3+(np.shape(self.X)[1])*0.025,-0.2+(i+1)*(-0.03),r'$\Delta_{'+str(Pg0.Data[5][i-Pg0.NumArtefacts])+'}$',fontsize = 12)
                                          
                    else:
                        self.axes.text(0.3+(np.shape(self.X)[1])*0.025,-0.2+(i+1)*(-0.03),r'$KCRV_{link}$',fontsize = 12)
                                   
                self.axes.text(0.3+(np.shape(self.X)[1])*0.025+0.1,-0.3,r'$+\mathrm{\varepsilon}$',fontsize = 12)             


                self.box.Add(self.canvas,1,wx.EXPAND)
                self.box.Fit(self)

            self.rank = linalg.matrix_rank(self.X)

            if (Pg0.editCompType.GetValue() == 'Linked' and self.rank < self.X.shape[1]) or (Pg0.editCompType.GetValue() == 'Primary' and self.rank < (self.X.shape[1] - 1)):
                self.RankDeficiency.ShowModal()
                self.GetParent().EnableTab(4,False)
                
            else:
                try:
                    B = linalg.cholesky(self.CovMatrix)
                    
                except:
                    self.CovMatError.ShowModal()
                    self.GetParent().EnableTab(4,False)
                    
                else:
                    if Pg0.editCompType.GetValue() == 'Linked':
                        self.GetParent().EnableTab(4,True)
                        
                    else:
                        if (sum(Pg1.Data[0])>0.95 and sum(Pg1.Data[0])<1.05):
                            self.GetParent().EnableTab(4,True)
  
                        else:
                            self.WeightsError.ShowModal()
                            self.GetParent().EnableTab(4,False)

In [8]:
class PageFour(scrolled.ScrolledPanel):
    # Set up page four where results are calculated and shown
    
    def __init__(self, parent):
        scrolled.ScrolledPanel.__init__(self, parent)

        Pg0 = self.GetParent().GetPage(0)
        Pg1 = self.GetParent().GetPage(1)
        Pg2 = self.GetParent().GetPage(2)
        Pg3 = self.GetParent().GetPage(3)
        
        self.box = wx.BoxSizer(wx.VERTICAL)
        self.SetSizer(self.box)
        self.Link = 0
        
    def PageChange(self):
        for i in range(0,len(self.GetChildren())):
            self.GetChildren()[0].Destroy()
            
        Pg0 = self.GetParent().GetPage(0)
        Pg1 = self.GetParent().GetPage(1)
        Pg2 = self.GetParent().GetPage(2)
        Pg3 = self.GetParent().GetPage(3)

        if Pg0.editCompType.GetValue() == 'Linked':
            # Calculate gamma, beta, U for linked comparison
            self.gamma1 = linalg.inv(Pg3.X.T*linalg.inv(Pg3.CovMatrix)*Pg3.X)
            self.beta = self.gamma1*Pg3.X.T*linalg.inv(Pg3.CovMatrix)*Pg3.y
            self.U = self.gamma1
            
        else:
            # Calculate c,w,gamma,beta,U for primary comparison
            self.c = 1/Pg3.CovMatrix.mean()
            self.wlist = ([0 for i in range(0, Pg0.NumArtefacts)])
            
            for i in range(0,Pg0.NumParticipants):
                self.wlist.append(Pg1.Data[0][i])
                
            self.w = np.matrix(self.wlist)
            self.gamma = linalg.inv(Pg3.X.T*linalg.inv(Pg3.CovMatrix)*Pg3.X+self.c*self.w.T*self.w)*Pg3.X.T*linalg.inv(Pg3.CovMatrix)
            self.beta = self.gamma*Pg3.y
            self.U = self.gamma*Pg3.CovMatrix*self.gamma.T
        
        # Calculate ChiSq, v
        self.chisq = (Pg3.y-Pg3.X*self.beta).T*linalg.inv(Pg3.CovMatrix)*(Pg3.y-Pg3.X*self.beta)
        print("chisq = ", self.chisq[0,0])
        self.v = Pg3.X.shape[0] - Pg3.X.shape[1]
        colour = self.GetParent().GetBackgroundColour()

        self.LabelA = wx.StaticText(self, -1, "Artefact Values", wx.Point(20,20))
        self.gridA = wx.grid.Grid(self, 10, name="Artefact Values", pos=(20,50), size = (350,500))
        self.gridA.SetDefaultRowSize(35,resizeExistingRows=False)
        self.gridA.SetDefaultColSize(80,resizeExistingCols=False)
        self.gridA.CreateGrid(Pg0.NumArtefacts,2)
        self.gridA.SetColLabelValue(0,'Value')
        self.gridA.SetColLabelValue(1,'Standard \n Uncertainty')
        self.gridA.SetDefaultCellBackgroundColour(colour)

                  
        for i in range(0,Pg0.NumArtefacts):
            self.gridA.SetRowLabelValue(i,str(Pg0.Data[4][i]))
            self.gridA.SetCellValue(i,0,"%1.4g" % self.beta[i])
            self.gridA.SetReadOnly(i,0)
            self.gridA.SetCellValue(i,1,"%1.4g" % np.sqrt(self.U[i,i]))
            self.gridA.SetReadOnly(i,1)
                      
        self.LabelP = wx.StaticText(self, -1, "Degrees of Equivalence", wx.Point(400,20))
        self.gridP = wx.grid.Grid(self, 10, name="Participant Biases", pos=(400,50),size=(350,500))
        self.gridP.SetDefaultRowSize(35,resizeExistingRows=False)
        self.gridP.SetDefaultColSize(80,resizeExistingCols=False)
        self.gridP.CreateGrid(Pg0.NumParticipants,2)
        self.gridP.SetColLabelValue(0,'Value')
        self.gridP.SetColLabelValue(1,'Standard \n Uncertainty')
        self.gridP.SetDefaultCellBackgroundColour(colour)


        for i in range(0,Pg0.NumParticipants):
            self.gridP.SetRowLabelValue(i,str(Pg0.Data[5][i]))
            self.gridP.SetCellValue(i,0,"%1.4g" % self.beta[i+Pg0.NumArtefacts])
            self.gridP.SetReadOnly(i,0)
            self.gridP.SetCellValue(i,1,"%1.4g" % np.sqrt(self.U[i+Pg0.NumArtefacts,i+Pg0.NumArtefacts]))
            self.gridP.SetReadOnly(i,1)

In [9]:
class MainWindow(wx.Frame):
    # Set up main window
    
    def __init__(self,parent,title):
        # Based on a frame, so set up the frame
        wx.Frame.__init__(self,parent,wx.ID_ANY, title)

        self.CreateStatusBar() # A Statusbar in the bottom of the window
        p = wx.Panel(self)
        bookstyle = aui.AUI_NB_TOP
        bookstyle&=~(aui.AUI_NB_SCROLL_BUTTONS)
        self.nb = aui.AuiNotebook(p,style=bookstyle)
        self.NTClicks=0
        
        # Page titles
        page0=PageZero(self.nb) # set up page 0
        self.nb.AddPage(page0, "Comparison Information")

        page1=PageOne(self.nb) # set up page 1
        self.nb.AddPage(page1, "Constraint")
        self.nb.EnableTab(1,False)
        
        page2=PageTwo(self.nb) # set up page 2
        self.nb.AddPage(page2, "Measurements")
        self.nb.EnableTab(2,False)

        page3=PageThree(self.nb) # set up page 3
        self.nb.AddPage(page3, "Matrices")
        self.nb.EnableTab(3,False)

        page4=PageFour(self.nb) # set up page 4
        self.nb.AddPage(page4, "Results" )
        self.nb.EnableTab(4,False)

        self.Bind(aui.EVT_AUINOTEBOOK_PAGE_CHANGING, self.OnTabChanged,)
        
        # Setting up the menus. 
        self.filemenu = wx.Menu()
        self.calcmenu = wx.Menu()
        self.helpmenu = wx.Menu()

        # The & character indicates the short cut key
        self.filemenu.Append(ID_OPEN, "&Open"," Open a file to edit")
        self.filemenu.AppendSeparator()
        self.filemenu.Append(ID_SAVE, "&Save"," Save file")
        self.filemenu.AppendSeparator()
        self.helpmenu.Append(ID_ABOUT, "&About"," Information about this program")
        self.filemenu.AppendSeparator()
        self.filemenu.Append(ID_EXIT,"E&xit"," Terminate the program")
        """
        self.calcmenu.Append(400,"Check Matrices","Construct Design Matrix and display all data")
        self.calcmenu.Enable(400,False)
        self.calcmenu.Append(500,"Calculate","Calculate the artefact values and participant biases")
        self.calcmenu.Enable(500,False)
        """

        # Creating the menubar.
        menuBar = wx.MenuBar()
        menuBar.Append(self.filemenu,"&File") # Adding the "filemenu" to the MenuBar
        #menuBar.Append(self.calcmenu,"&Calculate") # Adding the "calcmenu" to the MenuBar
        menuBar.Append(self.helpmenu,"&Help") # Adding the "helpmenu" to the MenuBar
        self.SetMenuBar(menuBar) # Adding the MenuBar to the Frame content.

        # Define the code to be run when a menu option is selected
        wx.EVT_MENU(self, ID_ABOUT, self.OnAbout)
        wx.EVT_MENU(self, ID_EXIT, self.OnExit)
        wx.EVT_MENU(self, ID_OPEN, self.OnOpen)
        #wx.EVT_MENU(self, 400, self.OnMatrix)
        #wx.EVT_MENU(self, 500, self.OnCalculate)
        wx.EVT_MENU(self, ID_SAVE, self.OnSave) 
        
        self.sizer=wx.BoxSizer()
        self.sizer.Add(self.nb,1,wx.EXPAND)
        self.x = 850
        self.y = 600
        self.SetClientSize(wx.Size(self.x, self.y)) 

        # Tell it which sizer is to be used for main frame
        # It may lay out automatically and be altered to fit window
        
        p.SetAutoLayout(1)
        p.SetSizer(self.sizer)
        p.Fit()
        
        # Show it !!!
        self.Show(1)

        # Define widgets early even if they're not going to be seen so that they can come up FAST when someone clicks for them!
        self.aboutme = wx.MessageDialog( self, " This tool for comparison analysis \n"
                                               " was created to make use of GLS accessible. \n"
                                               " Annette Koo, January 2012",
                            "About Comparison Analysis Tool", wx.OK)
        self.doiexit = wx.MessageDialog( self, " Exit - Are You Sure? \n",
                        "Exiting", wx.YES_NO)
        self.dirname = ''
        
    def OnTabChanged(self,evt):
        """
        Callback for aui.EVT_AUINOTEBOOK_PAGE_CHANGED
        """
        IntegersReq = wx.MessageDialog( self, " Please make sure that the numbers of measurements made by participants \n input on the 'Comparison Structure' tab are integers.",
                                        "Integers Required", wx.OK)

        if evt.GetSelection() == 1:
            try:
                for i in range(0,self.nb.GetPage(0).NumParticipants):
                    for j in range(0,self.nb.GetPage(0).NumArtefacts):
                        int(vars(self.nb.GetPage(0))['NumMeas'+"%02d" % i+"%02d" % j].GetValue())
                        
            except ValueError:
                IntegersReq.ShowModal()
                evt.Veto()
                
            else:
                self.nb.GetPage(1).PageChange()
               
        if evt.GetSelection() == 2:
            try:
                for i in range(0,self.nb.GetPage(0).NumParticipants):
                    for j in range(0,self.nb.GetPage(0).NumArtefacts):
                        int(vars(self.nb.GetPage(0))['NumMeas'+"%02d" % i+"%02d" % j].GetValue())
                        
            except ValueError:
                IntegersReq.ShowModal()
                evt.Veto()
                
            else:
                self.nb.GetPage(2).PageChange()
                self.nb.EnableTab(3,True)

        if evt.GetSelection() == 3:
            self.nb.GetPage(1).PageChange()
            self.nb.GetPage(2).PageChange()
            self.nb.GetPage(3).PageChange()

        if evt.GetSelection() == 4:
            self.nb.GetPage(1).PageChange()
            self.nb.GetPage(2).PageChange()
            self.nb.GetPage(3).PageChange()
            self.nb.GetPage(4).PageChange()

    def OnAbout(self,e):
        # A modal to describe comparison analysis tool
        self.aboutme.ShowModal() # Shows it

    def OnMatrix(self,e):
        # A modal to display the relevant matrices
        self.nb.GetPage(1).PageChange()
        self.nb.GetPage(2).PageChange()
        self.nb.GetPage(3).PageChange()
        self.nb.EnableTab(3,True)
           
    def OnCalculate(self,e):
        # A modal to display the relevant matrices
        self.nb.GetPage(1).PageChange()
        self.nb.GetPage(2).PageChange()
        self.nb.GetPage(3).PageChange()
        self.nb.GetPage(4).PageChange()
        self.nb.EnableTab(4,True)
           
    def OnExit(self,e):
        # A modal with an "are you sure" check
        igot = self.doiexit.ShowModal() # Shows it
        if igot == wx.ID_YES:
            self.Close(True) 

    def OnOpen(self,e):
        # Open an excel file, check the format, and put data values into notebook tabs
        dlg = wx.FileDialog(self, "Choose a file", self.dirname, "", "*.xls", wx.OPEN)
        
        if dlg.ShowModal() == wx.ID_OK:
            self.filename=dlg.GetFilename()
            self.dirname=dlg.GetDirectory()
            
            # Open the file, read the contents and set them into
            wb=xlrd.open_workbook(self.dirname+'\\'+self.filename)
            Data = wb.sheet_by_name('Control')
            Data0 = wb.sheet_by_name('Comparison Structure')
            Data1 = wb.sheet_by_name('Constraint')
            Data2 = wb.sheet_by_name('Measurements')
            Data3 = wb.sheet_by_name('Matrices')
            Data4 = wb.sheet_by_name('Results')
            
            Pg0 = self.nb.GetPage(0)
            Pg0.editTitl.SetValue(Data0.col_values(1)[0])
            Pg0.editCompType.SetValue(Data0.col_values(1)[1])
            Pg0.OnClick(wx.CommandEvent(wx.wxEVT_COMMAND_COMBOBOX_SELECTED,23))  
            Pg0.editNumArte.SetValue(int(Data0.col_values(1)[2]))
            Pg0.OnClick(wx.CommandEvent(wx.wxEVT_COMMAND_SPINCTRL_UPDATED,21)) 
            Pg0.editNumPart.SetValue(int(Data0.col_values(1)[3]))
            Pg0.OnClick(wx.CommandEvent(wx.wxEVT_COMMAND_SPINCTRL_UPDATED,22))

            for i in range(0,Pg0.NumArtefacts):
                vars(Pg0)['editArNa'+"%02d" % i].SetValue(Data0.col_values(1+i)[6])
                
                for j in range(0,Pg0.NumParticipants):
                    vars(Pg0)['editPaNa'+"%02d" % j].SetValue(str(Data0.row_values(7+j)[0]))
                    vars(Pg0)['NumMeas'+"%02d" % j+"%02d" % i].SetValue(str(int(Data0.row_values(7+j)[1+i])))
                    
            if Data.col_values(1)[2]:
                self.nb.EnableTab(1,True)
               
                Pg1 = self.nb.GetPage(1)
                self.nb.GetPage(1).PageChange()
                
                if Pg0.editCompType.GetValue() == 'Primary':
                    for i in range(0,Pg0.NumParticipants):
                        vars(Pg1)['editWeights'+"%02d" % i].SetValue(str(Data1.col_values(1)[1+i]))
                        
                if Pg0.editCompType.GetValue() == 'Linked':
                    Pg1.editNumLink.SetValue(str(int(Data1.col_values(1)[0])))
                    Pg1.On2Click(wx.CommandEvent(wx.wxEVT_COMMAND_COMBOBOX_SELECTED,50))
                    
                    for i in range(1,int(Pg1.editNumLink.GetValue())+1):
                        vars(Pg1)['Link'+"%02d" % i].SetValue(str(Data1.row_values(1)[i]))
                        Pg1.On2Click(wx.CommandEvent(wx.wxEVT_COMMAND_COMBOBOX_SELECTED,51+i))
                        
                    Pg1.editLinkKCRV.SetValue(str(Data1.col_values(1)[2]))
                    Pg1.On2Click(wx.CommandEvent(wx.wxEVT_COMMAND_COMBOBOX_SELECTED,90))
                          
            if Data.col_values(1)[3]:
                self.nb.EnableTab(2,True)
                Pg2 = self.nb.GetPage(2)
                self.nb.GetPage(2).PageChange()
                
                if Pg0.editCompType.GetValue() == 'Primary':
                    Test = 'No'
                    
                else:
                    Test = Pg1.editLinkKCRV.GetValue()
                    
                for i in range(0,Pg2.NMeasurements):       
                    if i != Pg2.NMeasurements-1 or Test != 'Yes':
                        Pg2.myGrid.SetCellValue(i+1,0,str(Data2.col_values(0)[1+i]))
                        Pg2.MeasurementValues[i] = (Pg2.myGrid.GetCellValue(i+1,0))
                                 
                        #vars(Pg2)['MeasValue'+"%02d" % i].SetValue(str(Data2.col_values(0)[1+i]))
                        
                    for j in range(0,Pg2.NMeasurements):
                        Pg2.myGrid.SetCellValue(i+1,j+1,str(Data2.col_values(4+i)[1+j]))
                        Pg2.CovarianceMatrix[i][j] = Pg2.myGrid.GetCellValue(i+1,j+1)
                        print(i)
                        print(j)
                        #vars(Pg2)['Cov'+"%02d" % j+"%02d" % i].SetValue(str(Data2.col_values(4+i)[1+j]))

            if Data.col_values(1)[4]:
                self.nb.EnableTab(3,True)
                self.nb.GetPage(3).PageChange()

            if Data.col_values(1)[5]:
                self.nb.EnableTab(4,True)
                self.nb.GetPage(4).PageChange()                   
                   
            # Report on name of latest file read
            self.SetTitle("Editing ... "+self.filename)

        dlg.Destroy()

    def OnSave(self,e):
        # Save away the edited text
        # Open the file, do an RU sure check for an overwrite
        dlg = wx.FileDialog(self, "Choose a file", self.dirname, "", "*.xls", \
                wx.SAVE | wx.OVERWRITE_PROMPT)
        
        if dlg.ShowModal() == wx.ID_OK:               
            # update all pages before saving data by forcing a 'page change' event for each    
            self.nb.GetPage(1).PageChange()
            self.nb.GetPage(2).PageChange()
            # Open the file for write, write, close
            self.filename=dlg.GetFilename()
            self.dirname=dlg.GetDirectory()
            
            wb = xlwt.Workbook()
            SheetLbls = np.array(['Control', 'Comparison Structure', 'Constraint', 'Measurements', 'Matrices','Results'])

            for k in range(0,6):
                ws = wb.add_sheet(SheetLbls[k])       
                if k == 0:
                    ws.write(0,0,"Options Enabled")
                    ws.write(1,0,SheetLbls[1])
                    ws.write(1,1,self.nb.GetPage(0).IsEnabled())
                    ws.write(2,0,SheetLbls[2])
                    ws.write(2,1,self.nb.GetPage(1).IsEnabled())
                    ws.write(3,0,SheetLbls[3])
                    ws.write(3,1,self.nb.GetPage(2).IsEnabled())
                    ws.write(4,0,SheetLbls[4])
                    ws.write(4,1,self.nb.GetPage(3).IsEnabled())
                    ws.write(5,0,SheetLbls[5])
                    ws.write(5,1,self.nb.GetPage(4).IsEnabled())

                if k == 1 and self.nb.GetPage(0).IsEnabled() == True:
                    Data = self.nb.GetPage(0).Data
                    ws.write(0,0,'Title')
                    ws.write(0,1,Data[0])
                    ws.write(1,0,"Comparison Type")
                    ws.write(1,1,Data[1])
                    ws.write(2,0,'Number of Artefacts')
                    ws.write(2,1,Data[2])
                    ws.write(3,0,'Number of Participants')
                    ws.write(3,1,Data[3])
                    ws.write(4,0, 'Number of Measurements')
                    ws.write(5,1,'Artefacts')
                    
                    for i in range(0,Data[2]):
                        ws.write(6,1+i,Data[4][i])
                        
                    ws.write(6,0,'Participants')
                    
                    for i in range(0,Data[3]):
                        ws.write(7+i,0,Data[5][i])
                        
                    for i in range(0,Data[3]):
                        for j in range(0,Data[2]):
                            ws.write(7+i,1+j,Data[6][i][j]) 
                            
                if k == 2 and self.nb.GetPage(1).IsEnabled() == True:
                    Data = self.nb.GetPage(1).Data
                    
                    if self.nb.GetPage(0).Data[1] == 'Linked':
                        ws.write(0,0,'Number of Linking Labs')
                        ws.write(0,1,Data[0])
                        ws.write(1,0,'Linking Labs')
                        
                        for i in range(0,Data[0]):
                            ws.write(1,1+i,Data[1][i])
                            
                        ws.write(2,0,'Include KCRV')
                        ws.write(2,1,Data[2])
                        
                    else:
                        ws.write(0,0,'Weighting')
                        
                        for i in range(0,self.nb.GetPage(0).Data[3]):
                            ws.write(1+i,0,self.nb.GetPage(0).Data[5][i])
                            
                            if Data[0][i] == '':
                                ws.write(1+i,1,'0.0')
                                
                            else:
                                ws.write(1+i,1,Data[0][i])
                                     
                if k == 3 and self.nb.GetPage(2).IsEnabled() == True:
                    Data = self.nb.GetPage(2).Data
                    ws.write(0,0,'Measured Value')
                    ws.write(0,4,'Covariance Matrix')
                    
                    for i in range(0,len(Data[1])):
                        ws.write(1+i,1,Data[0][i][0])
                        ws.write(1+i,2,Data[0][i][1])
                        ws.write(1+i,3,Data[0][i][2])
                        ws.write(1+i,0,Data[1][i])
                        
                        for j in range(0,len(Data[1])):
                            ws.write(1+i,4+j,Data[2][i][j])

                if k == 4 and self.nb.GetPage(3).IsEnabled() == True and self.nb.GetPage(4).IsEnabled() == True:
                    Pg0 = self.nb.GetPage(0)
                    Pg2 = self.nb.GetPage(2)
                    Pg3 = self.nb.GetPage(3)
                    
                    ws.write(0,0,'Measured Values')
                    ws.write(1,1,'=')
                    ws.write(0,2,'X matrix')
                    ws.write(0,3+np.shape(Pg3.X)[1],'Unknowns')
                    
                    for i in range(0,len(Pg2.Data[1])):
                        ws.write(1+i,0,Pg2.Data[1][i])
                        for j in range(0,np.shape(Pg3.X)[1]):
                            ws.write(1+i,2+j,Pg3.X[i,j])
                            
                            if i == 0:
                                if j < Pg0.NumArtefacts:
                                    ws.write(1+j,3+np.shape(Pg3.X)[1], 'theta_'+str(Pg0.Data[4][j]))
                                    
                                elif j < Pg0.NumArtefacts + Pg0.NumParticipants:
                                    ws.write(1+j,3+np.shape(Pg3.X)[1],'Delta_'+str(Pg0.Data[5][j-Pg0.NumArtefacts]))
                                            
                                else:
                                    ws.write(1+j,3+np.shape(Pg3.X)[1], 'KCRV_link')
                                    
                if k == 5 and self.nb.GetPage(4).IsEnabled() == True:

                    Pg0 = self.nb.GetPage(0)
                    Pg4 = self.nb.GetPage(4)
                    
                    ws.write(0,0,'Unknowns')
                    ws.write(0,1,'Values')
                    ws.write(0,3,'Covariance Matrix')
                    
                    for i in range(0,len(Pg4.beta)):
                        if i < Pg0.NumArtefacts:
                            ws.write(1+i,0, 'theta_'+str(Pg0.Data[4][i]))
                              
                        elif i < Pg0.NumArtefacts + Pg0.NumParticipants:
                            ws.write(1+i,0,'Delta_'+str(Pg0.Data[5][i-Pg0.NumArtefacts]))
                              
                        else:
                            ws.write(1+i,0, 'KCRV_link')
                              
                        ws.write(1+i,1,Pg4.beta[i,0])
                        
                        for j in range(0,len(Pg4.beta)):
                            ws.write(1+i,3+j,Pg4.U[i,j])
                              

                       
                       
            wb.save(self.dirname+'\\'+self.filename)
        dlg.Destroy()

In [11]:
app = wx.App()
view = MainWindow(None, "Comparison Analysis")

# Enter event loop
app.MainLoop()

C:\Users\e.molloy.IRL\Documents\Ellie\envs\Ellie\lib\site-packages\ipykernel_launcher.py:16: wxPyDeprecationWarning: Call to deprecated item __call__. Use :meth:`EvtHandler.Bind` instead.
  app.launch_new_instance()
C:\Users\e.molloy.IRL\Documents\Ellie\envs\Ellie\lib\site-packages\ipykernel_launcher.py:23: wxPyDeprecationWarning: Call to deprecated item __call__. Use :meth:`EvtHandler.Bind` instead.
C:\Users\e.molloy.IRL\Documents\Ellie\envs\Ellie\lib\site-packages\ipykernel_launcher.py:33: wxPyDeprecationWarning: Call to deprecated item __call__. Use :meth:`EvtHandler.Bind` instead.
C:\Users\e.molloy.IRL\Documents\Ellie\envs\Ellie\lib\site-packages\ipykernel_launcher.py:42: wxPyDeprecationWarning: Call to deprecated item __call__. Use :meth:`EvtHandler.Bind` instead.
C:\Users\e.molloy.IRL\Documents\Ellie\envs\Ellie\lib\site-packages\ipykernel_launcher.py:64: wxPyDeprecationWarning: Call to deprecated item __call__. Use :meth:`EvtHandler.Bind` instead.
C:\Users\e.molloy.IRL\Documents\

0